In [2]:
import polars as pl
import os
import duckdb as db

# Chemin data

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data = os.path.join(parent_dir, "data")
data_path = os.path.join(data, "BPE23.parquet")

# BDD

In [4]:
df = pl.scan_parquet(data_path)

In [7]:
df_filtered = df.filter(
    (pl.col("DOM") == "D") |
    (pl.col("SDOM").is_in(["D1", "D2", "D3"])) |
    (pl.col("TYPEQU").is_in(["D101", "D102", "D103", "D108", "D113"])) |
    (pl.col("ACCES_SANITAIRE") == "1") |
    (pl.col("PRES_SANITAIRE") == "1")
).select([
    "DOM", "SDOM", "TYPEQU", "CAPACITE", "ACCES_SANITAIRE", "PRES_SANITAIRE",
    "DENS3", "DENS7", "AAV2020", "QP", "QP2015", "LATITUDE", "LONGITUDE",
    "CODPOS", "LIBCOM"
]).collect()


In [8]:
df_filtered

DOM,SDOM,TYPEQU,CAPACITE,ACCES_SANITAIRE,PRES_SANITAIRE,DENS3,DENS7,AAV2020,QP,QP2015,LATITUDE,LONGITUDE,CODPOS,LIBCOM
str,str,str,f64,str,str,str,str,str,str,str,f64,f64,str,str
"""D""","""D1""","""D101""",null,"""_Z""","""_Z""","""2""","""2""","""9A1""","""IND""","""QP971014""",16.232796,-61.529615,"""97139""","""LES ABYMES"""
"""D""","""D1""","""D101""",null,"""_Z""","""_Z""","""2""","""2""","""9A1""","""IND""","""QP971014""",16.236421,-61.528087,"""97139""","""LES ABYMES"""
"""D""","""D1""","""D101""",null,"""_Z""","""_Z""","""2""","""2""","""9A1""","""IND""","""QP971014""",16.235507,-61.529595,"""97139""","""LES ABYMES"""
"""D""","""D1""","""D102""",null,"""_Z""","""_Z""","""2""","""2""","""9A1""","""IND""","""HZ""",16.266926,-61.510634,"""97139""","""LES ABYMES"""
"""D""","""D1""","""D102""",null,"""_Z""","""_Z""","""2""","""2""","""9A1""","""IND""","""QP971014""",16.236421,-61.528087,"""97139""","""LES ABYMES"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""D""","""D2""","""D201""",null,"""_Z""","""_Z""","""3""","""6""","""104""","""CSZ""","""CSZ""",42.508419,9.370341,"""20290""","""VOLPAJOLA"""
"""D""","""D5""","""D505""",null,"""_Z""","""_Z""","""3""","""6""","""104""","""CSZ""","""CSZ""",42.510327,9.352729,"""20290""","""VOLPAJOLA"""
"""F""","""F1""","""F118""",null,"""0""","""1""","""3""","""6""","""104""","""CSZ""","""CSZ""",42.50685,9.37334,"""20290""","""VOLPAJOLA"""


Les équipements de santé sont dans les variables :
    - DOM : D
    - SDOM : D1, D2, D3
    - TYPEQU : D101, D102, D103, D108, D113

variables importantes :
    - DOM  
    - SDOM 
    - TYPEQU 
    - CAPACITE 
    - ACCES_SANITAIRE 
    - PRES_SANITAIRE 
    - DENS3 
    - DENS7
    - AAV2020 
    - QP & QP2015
    - LATITUDE & LONGITUDE 
    - CODPOS 
    - LIBCOM 

# DuckDB

In [4]:
df = pl.scan_parquet(data_path)

In [5]:
df_filtered = df.select([
    "DOM", "SDOM", "TYPEQU", "CAPACITE", "ACCES_SANITAIRE", "PRES_SANITAIRE",
    "DENS3", "DENS7", "AAV2020", "QP", "QP2015", "LATITUDE", "LONGITUDE",
    "CODPOS", "LIBCOM"
])


In [6]:
df_filtered = df_filtered.collect()

In [7]:
con = db.connect(database=':memory:')
con.register('df', df_filtered)

In [8]:
query = """
SELECT *
FROM df
WHERE
    DOM = 'D'
    OR SDOM IN ('D1', 'D2', 'D3')
    OR TYPEQU IN ('D101', 'D102', 'D103', 'D108', 'D113')
    OR ACCES_SANITAIRE = '1'
    OR PRES_SANITAIRE = '1'
"""

In [9]:
df_sante = con.execute(query).fetchdf()

: 